<a></a>
# ResNet - Skin Lesions Image Classification with ISIC 2019

Access Drive which contains the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Variables Initialization

In [ ]:
# Train fcc
second = False
epochs = 60 #sse mudo aqui tenho que mudar fcc o lr no_epochs=50
# freeze = false - means initialization but not freeze
freeze = False


### Filepath saved weights 
random = True # true = no weights loaded from the previous task

# Path where to save the baseline model
filepath_fcc = '/content/drive/MyDrive/Tese/fold4_baselineimagenet'
filepath_save_img = '/content/drive/MyDrive/Tese/fold4_baselineimagenet.pdf'

#----------------------------------
### Network 
weight_resnet = True #true =imagenet
res = 'old'
# res = 'new'
## Resnet50
rede = 'resnet50' 
h1 = 256
h2 = 128

#----------------------------------
### Evaluation method
aval = 'fcc'
extra_mlp = False # true if we want fcc after the gap and before softmax
h1_fcc = 512
h2_fcc = 128

# aval = 'svm'
param_c = 100
param_g = 0.1

#----------------------------------
### Feature Space
fspace = 'gap'  #2048 / 64
# fspace = 'h2' #128 / 32
# fspace = 'h1' #256 / 64



In [ ]:
if second == False:
    !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed_fold4.zip -d /content

<a></a>
## 1.Imports

In [ ]:
import csv
import pandas as pd
import numpy as np
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from collections import Counter

sys.path.append('/content/drive/MyDrive/Tese/')

from visuals import plot_grouped_2bars
import math
import shutil
from sklearn.utils import shuffle

import glob

import os, os.path
from ast import literal_eval
from make_dir_versao_writedisco import *

import cv2

from PIL import Image


import keras
from tensorflow.keras.layers import Input, Dense, Activation,Flatten, Convolution2D, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import layers, losses

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
#from tensorflow.python.keras.preprocessing.image import image_dataset_from_directory

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

from mpl_toolkits.mplot3d import Axes3D

from tensorflow.keras.models import Sequential, save_model, load_model

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV , StratifiedShuffleSplit
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report

import seaborn as sns 

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600
BATCH_SIZE = 32

<a></a>
## 2. Online Data Augmentation

In [ ]:
# from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input

datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        #rotation_range=90,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        # rescale=1./255,
        #shear_range=0.2,
        #brightness_range=[0.2,1],
        vertical_flip = True,
        horizontal_flip=True)

datagenV = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        #rescale=1./255
        )

seed = 1
train_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 2
validation_generator = datagenV.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 3
trainPredict_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 4
validationPredict_generator = datagenV.flow_from_directory(
    directory =  '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)


Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.
Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.


<a></a>
## 3.Build ResNet Architecture

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
if weight_resnet == True:
  MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
  print('\nloaded weights from imagenet')
  # MODEL.summary()
else:
  MODEL = ResNet50(include_top=False, weights= None, input_tensor=None, input_shape=(224,224,3), pooling=None)


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

model = tf.keras.Sequential(MODEL)
# Flatten the output layer to 1 dimension
model.add(GlobalAveragePooling2D())

model.add(Dense(256,name='dense_1'))
model.add(Dropout(0.5))
model.add(Activation("relu"))
model.add(Dense(128,name='dense_2'))

full_model_gap = Model(inputs=model.input, outputs=model.layers[-5].output)
# full_model_gap.summary()

model = Sequential()
model.add(full_model_gap)
model.add(Dropout(0.5)) #profs disseram que melhora

model.add(layers.Dense(8,activation = 'softmax'))
model.summary()
                                           
        
          
optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

<a></a>
## 3. Fit

In [ ]:
def lr_scheduler(epoch, lr):
    decay_rate = 0.3
    if epoch == 2 or epoch == 4 or epoch ==7 or epoch == 15  or epoch == 20:
        return lr*decay_rate 
    return lr

saveWeights = ModelCheckpoint(filepath_fcc, save_best_only=True, monitor='val_loss', mode='min')
learningRate = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, verbose=1,  mode='min')
#tensBoard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BATCH_SIZE, write_graph=True, write_grads=True, write_images=True, update_freq='epoch')



class_weights = {0: NUM_TRAIN_SAMPLES/3165,
                1: NUM_TRAIN_SAMPLES/9012,
                2: NUM_TRAIN_SAMPLES/2326,
                3: NUM_TRAIN_SAMPLES/607,
                4: NUM_TRAIN_SAMPLES/1837,
                5: NUM_TRAIN_SAMPLES/167,
                6: NUM_TRAIN_SAMPLES/177,
                7: NUM_TRAIN_SAMPLES/440}


# model.load_weights(filepath_fcc)
history = model.fit_generator(train_generator,
                    #steps_per_epoch = NUM_TRAIN_SAMPLES // BATCH_SIZE,
                    epochs = 35, verbose =1, 
                    validation_data = validation_generator,
                    shuffle = True,
                    workers = 4, 
                    #callbacks =[LearningRateScheduler(lr_scheduler, verbose=1),saveWeights,learningRate, tensBoard],
                    callbacks =[saveWeights,learningRate],
                    #validation_steps = NUM_VAL_SAMPLES // BATCH_SIZE,
                    class_weight = class_weights)
    


In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()




<a></a>
## 4. Analise the Trained model

In [ ]:
from tensorflow.keras.models import model_from_json

# load weights into new model
model.load_weights(filepath_fcc)

In [ ]:
print('train')
Y_pred = model.predict_generator(trainPredict_generator)
y_pred = np.argmax(Y_pred, axis=1)
print(len(y_pred))
print(len(trainPredict_generator.classes))
print('Confusion Matrix Validation')
validConfMatrix =confusion_matrix(trainPredict_generator.classes, y_pred)
print('Accuracy train: '+str(accuracy_score(trainPredict_generator.classes, y_pred)))
print(validConfMatrix)
print(classification_report(trainPredict_generator.classes, y_pred))#Output

diag = 0
vec_val = [1357,3863,997,260,787,72,76,188]
vec_train = [3165,9012,2326,607,1837,167,177,440]
for col in range(0, 8):
  for lin in range(0, 8):
      if col == lin:
        diag = (diag + validConfMatrix[lin][col]/vec_train[col])
diag=diag/8
print('Diagonal mean value:')
print(diag)

print('\nvalid')
Y_pred = model.predict_generator(validationPredict_generator)
y_pred = np.argmax(Y_pred, axis=1)
print(len(y_pred))
print(len(validationPredict_generator.classes))
print('Confusion Matrix Validation')
validConfMatrix =confusion_matrix(validationPredict_generator.classes, y_pred)
print('Accuracy valid: '+str(accuracy_score(validationPredict_generator.classes, y_pred)))
print(validConfMatrix)
print(classification_report(validationPredict_generator.classes, y_pred,digits=4))#Output

diag = 0
vec_val = [1357,3863,997,260,787,72,76,188]
vec_train = [3165,9012,2326,607,1837,167,177,440]
for col in range(0, 8):
  for lin in range(0, 8):
      if col == lin:
        diag = (diag + validConfMatrix[lin][col]/vec_val[col])
diag=diag/8
print('Diagonal mean value:')
print(diag)


In [ ]:
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    plt.clf()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Ground Truth')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    if normalize:
        plt.savefig(filepath_save_img,dpi=300, bbox_inches='tight')
    else:
        plt.savefig(filepath_save_img,dpi=300, bbox_inches='tight')  

    
                   
cm_plot_labels = ['MEL', 'NV', 'BCC', 'AK', 'BKL','DF', 'VASC', 'SCC']
plot_confusion_matrix(validConfMatrix, cm_plot_labels)
plot_confusion_matrix(validConfMatrix, cm_plot_labels, True)

In [ ]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, recall_score, precision_score
balanced_accuracy_score(validationPredict_generator.classes, y_pred, sample_weight=None, adjusted=False)

In [ ]:
recall_score(validationPredict_generator.classes, y_pred, average=None)

In [ ]:
FP = [0,0,0,0,0,0,0,0]
TN = [0,0,0,0,0,0,0,0]
SP = [0,0,0,0,0,0,0,0]
        
for lesion in range(0,8):
    for i in range(0, 8):
        if i != lesion:
            FP[lesion] = FP[lesion] + int(validConfMatrix[i][lesion])

    for i in range(0, 8):
        for j in range(0, 8):
            if i != lesion and j != lesion:
                TN[lesion] = TN[lesion] + int(validConfMatrix[i][j])

for i in range(0,8):
    SP[i] = TN[i]/(TN[i]+FP[i])

In [ ]:
SP

In [ ]:
sum(SP)/len(SP)

In [ ]:
precision_score(validationPredict_generator.classes, y_pred, average='macro')